The contents of this script are independently licensed under the Creative Commons Attribution 4.0 International License (CC BY 4.0). This licensing applies solely to this script and does not affect the licensing terms of the source repository, should they differ. © 2024 Luca Kunz.

# Read the TRAPS DRIFTERS HPS dataset

Load the dataset of 20-years (hyperbolic) pair statistics of drogued and undrogued drifter-TRAP pairs.
One can use this 20-years TRAPS DRIFTERS HPS dataframe together with the yearly TRAPS GPGP dataframes to reproduce our analysis.

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import time
import datetime
import pickle

from IPython.display import display, Audio
# import jupyter notebook files like regular python modules
import import_ipynb
from aa_define_classes import TRAPSdata

importing Jupyter notebook from aa_define_classes.ipynb


In [2]:
# measure the computation time for the entire script
start_script_timer = time.perf_counter()

# Preliminary

## Pickle object saving function

In [3]:
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

## Files and paths

In [4]:
# if script is running in jupyter lab
if sys.argv[0].endswith("ipykernel_launcher.py"):
    # set the velocity product
    vel_product_ID = 1
    epsilon_ID = 1
    rho_ID = 1
    notebook_run = True
#    save_fig = True
    save_fig = False

    
# if script is running as python script
else:
    # read in product from bash
    vel_product_ID = int(sys.argv[1])
    # read in epsilon from bash
    epsilon_ID = int(sys.argv[2])
    # read in rho from bash
    rho_ID = int(sys.argv[3])
    notebook_run = False
    save_fig = True


vel_product_short = ['ENSRYS_24HM', 'MULTIOBS_24HI', 'MULTIOBS_24HM', 'SEALEVEL_24HI'][vel_product_ID]

vel_product_long = ['CMEMS GLOBAL_REANALYSIS_PHY_001_031 ENSEMBLE MEAN (1/4°, 24HM)', 
                    'CMEMS MULTIOBS_GLO_PHY_REP_015_004 (1/4°, 24HI)', 
                    'CMEMS MULTIOBS_GLO_PHY_REP_015_004 (1/4°, 24HM)', 
                    'CMEMS SEALEVEL_GLO_PHY_L4_MY_008_047 (1/4°, 24HI)'][vel_product_ID]


epsilon_value = [0.1, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0][epsilon_ID]
epsilon_string = ['_e010', '_e025', '_e050', '_e075', '_e100', '_e125', '_e150', '_e175', '_e200'][epsilon_ID]

rho_kmvalue = [50, 75, 100, 150, 200, 250, 300][rho_ID] # in kilometres
rho_kmstring = ['_r050', '_r075', '_r100', '_r150', '_r200', '_r250', '_r300'][rho_ID] # in kilometres

gdp_product_ID = 2 # set this a priori
gdp_product_short = ['GDP_1HI', 'GDP_6HI', 'GDP_24HI'][gdp_product_ID]

In [5]:
# define the path to the pkl files
pkl_HPS_importpath = 'export_pkl/' + vel_product_short + '/'

# Import DataFrames

## Load HPS data

We load the dataset of 20-years (hyperbolic) pair statistics of drogued and undrogued drifter-TRAP pairs. Each row in the dataframe corresponds to one detected and tracked drifter-TRAP pair. Some columns involve arrays instead of single values to indicate the respective attribute value at every instance of the corresponding drifter-TRAP pair. The loading of the dataframe only takes approx. 2GB of memory. The following lists all pair `attributes` with a short description:

`pair_ID`: the label/identifier of a drifter-TRAP pair which is composed of the timestamp of first encounter, the drifter_ID and the origin_ID of the closest TRAP

`origin_ID`: the label/identifier of the closest TRAP

`retention_time`: the number of days the drifter stays around its closest TRAP

`proximity_time`: the number of hours the drifter stays around its closest TRAP

`TRAP_lifetime`: the number of days for which the closest TRAP exists (irrespective of the drifter encounter)

`minimum_TRAP_age`: the age of the closest TRAP at first encounter

`reachable`: indicator if the drifter can reach the given TRAP core before it dissipates, considering an average drifter speed of 17 km/day that we find in our dataset

`vircles_always_available`: indicator if all TRAP instances of the current pair have a vorticity circle defined around them

`involved_attractions`: the core attraction rates $s_1$ of the closest TRAP in chronological order

`involved_qorders`: the quadrupole orders of the vorticity patterns detected around the closest TRAP in chronological order (**no background vorticity**)

`involved_phase_shifts`: for every instance of the closest TRAP and in chronological order, the angle of the longer branch of the TRAP with the zonal axis 0° pointing Eastwards

`retention_nages`: a pair's age normalised by its retention time for pairs remaining longer than two days, the pair age at first encounter corresponds to 0%, the pair age at last encounter to 100%

`original_core_distances`: the distances \[km\] between the drifter and its closest TRAP core in chronological order 

`original_curve_distances`: the distances \[km\] between the drifter and the closest curve point of its closest TRAP in chronological order

`drogued`: indicator if the drifter is drogued

In [6]:
# define the pkl file to load
pkl_HPS_importname = vel_product_short + epsilon_string + '_' + gdp_product_short
pkl_HPS_importname += '_TRAPS_DRIFTERS_HPS_0019' + rho_kmstring + '.pkl'

# load the pickle object
with open(pkl_HPS_importpath + pkl_HPS_importname, 'rb') as inp:
    # cdf for current DataFrame, this will be overwritten every loop to save memory
    pd_HPS_df = pickle.load(inp).pd_TRAPS_df.copy()

# get the description of the dataset
with open(pkl_HPS_importpath + pkl_HPS_importname, 'rb') as inp: 
    hps_description = pickle.load(inp).product_long

# reset the index after loading
pd_HPS_df.reset_index(drop=True, inplace=True)

# Show DataFrame

In [7]:
pd_HPS_df

,pair_ID,origin_ID,retention_time,proximity_time,TRAP_lifetime,minimum_TRAP_age,reachable,vircles_always_available,involved_attractions,involved_qorders,involved_phase_shifts,retention_nages,original_core_distances,original_curve_distances,drogued
0,201310310000 000000000101916 201310270000 344,201310270000 344,46,1104,93,5,True,True,"[-0.139247397665006, -0.133322592384704, -0.14...","[2.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...","[4.430945874178001, 4.954220006808431, 5.01636...","[0.0, 0.022222222222222223, 0.0444444444444444...","[21.63602628438114, 27.218985418574956, 25.023...","[14.726172213721245, 22.42580327745963, 22.921...",True
1,201905270000 300234066316870 201905150000 291,201905150000 291,45,1080,81,13,True,True,"[-0.20999680205224, -0.196616953730887, -0.196...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.4909013430791804, 1.4696727712275444, 1.509...","[0.0, 0.022727272727272728, 0.0454545454545454...","[69.84518738773463, 46.86426584419498, 36.5879...","[69.67409225165218, 46.92543720394726, 36.4688...",True
2,201507280000 000000000132718 201507280000 374,201507280000 374,44,1056,57,1,True,True,"[-0.201265758720064, -0.19937420075606, -0.181...","[0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, ...","[5.207125086723592, 5.248221535516462, 5.36299...","[0.0, 0.023255813953488372, 0.0465116279069767...","[44.39999983897825, 30.620572475120394, 16.955...","[38.90789304323914, 21.04790099011558, 1.40130...",True
3,200010120000 000000009729746 200010120000 069,200010120000 069,41,984,88,1,True,True,"[-0.181595150679058, -0.205832111166168, -0.20...","[1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, ...","[3.432881234756834, 3.4329792126763725, 3.4690...","[0.0, 0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0....","[61.133880337996544, 55.559512408494975, 49.23...","[54.559969221516106, 51.084681066096465, 42.73...",True
4,201102160000 000000000093582 201101310000 284,201101310000 284,39,936,55,17,True,True,"[-0.107347613092045, -0.111451057209841, -0.12...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.322312406238966, 4.600036111789788, 4.46473...","[0.0, 0.02631578947368421, 0.05263157894736842...","[51.59612662718492, 25.10434105324435, 19.7163...","[51.596141667616735, 23.93646643677028, 18.867...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62339,201103250000 000000000083560 201103250000 437,201103250000 437,1,24,2,1,False,True,[-0.16899477078028],[1.0],[5.390509011270028],[],[58.11148631641262],[52.365780338880654],False
62340,201103250000 000000000070914 201101250000 054,201101250000 054,1,24,70,60,True,True,[-0.40083146221308],[0.0],[4.776045595492729],[],[65.05433042704217],[45.26822944327256],False
62341,201103250000 000000000063828 201103030000 065,201103030000 065,1,24,30,23,True,True,[-0.24310940143133],[0.0],[6.138525011551468],[],[54.92517681455478],[14.871330290592807],False
62342,201103240000 000000000090138 201103240000 241,201103240000 241,1,24,4,1,True,True,[-0.228641361413956],[1.0],[3.9887105766753943],[],[72.63984723215371],[45.75185209596553],False


# Show metadata

In [8]:
hps_description

'TRansient Attracting Profiles (TRAPs, Serra et al. (2020) published at https://doi.org/10.1038/s41467-020-16281-x) have been computed with code by Mattia Serra (https://github.com/MattiaSerra/TRAPs) and post-processed with code by Luca Kunz (https://github.com/kunzluca/trapsgpgp) using daily snapshots of near-surface geostrophic + Ekman currents from the product Global Total Surface and 15m Current (COPERNICUS-GLOBCURRENT) from Altimetric Geostrophic Current and Modeled Ekman Current Reprocessing that is provided by the E.U. Copernicus Marine Service (https://doi.org/10.48670/moi-00050). Surface drifter positions have been consulted from the Global Drifter Program (Lumpkin and Centurioni (2019) available at https://doi.org/10.25921/7ntx-z961). This work is a contribution to the project L3 (Meso- to submesoscale turbulence in the ocean) of the Collaborative Research Centre TRR 181 Energy Transfer in Atmosphere and Ocean funded by the German Research Foundation (DFG) and has been conduc

# End sound

In [9]:
# measure the computation time for the entire script
stop_script_timer = time.perf_counter()
print(f'overall computation time: {stop_script_timer - start_script_timer:0.3f} seconds')

overall computation time: 2.320 seconds


In [10]:
#https://gist.github.com/tamsanh/a658c1b29b8cba7d782a8b3aed685a24

framerate = 4410
play_time_seconds = 1

t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
# G-Dur
#audio_data = np.sin(2*np.pi*391*t) + np.sin(2*np.pi*493*t) + np.sin(2*np.pi*587*t)
# D-Dur
audio_data = np.sin(2*np.pi*293*t) + np.sin(2*np.pi*369*t) + np.sin(2*np.pi*440*t)
Audio(audio_data, rate=framerate, autoplay=True)